In [4]:
%pip install openmeteo-requests retry-requests requests-cache numpy pandas



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [requests-cache]m [cattrs]
Note: you may need to restart the kernel to use updated packages.


In [6]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# 1. Client Open-Meteo avec cache + retry
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"

idf_departements = [
    {"code": "75", "name": "Paris",            "lat": 48.8566, "lon": 2.3522},
    {"code": "92", "name": "Hauts-de-Seine",   "lat": 48.85,   "lon": 2.19},
    {"code": "93", "name": "Seine-Saint-Denis","lat": 48.91,   "lon": 2.31},
    {"code": "94", "name": "Val-de-Marne",     "lat": 48.78,   "lon": 2.49},
    {"code": "91", "name": "Essonne",          "lat": 48.46,   "lon": 2.16},
    {"code": "78", "name": "Yvelines",         "lat": 48.76,   "lon": 1.28},
    {"code": "95", "name": "Val-d'Oise",       "lat": 49.07,   "lon": 1.82},
    {"code": "77", "name": "Seine-et-Marne",   "lat": 48.65,   "lon": 2.96},
]

all_dfs = []

for dep in idf_departements:
    print(f" Récupération des données météo pour {dep['name']} (2024)...")

    params = {
        "latitude":  dep["lat"],
        "longitude": dep["lon"],
        "start_date": "2024-01-01",
        "end_date":   "2024-12-31",
        "hourly": [
            "temperature_2m",
            "wind_speed_10m",
            "cloud_cover",
            "direct_radiation",
            "diffuse_radiation"
        ],
        "timezone": "UTC"
    }

    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    # Numpy arrays
    temp    = hourly.Variables(0).ValuesAsNumpy()
    wind    = hourly.Variables(1).ValuesAsNumpy()
    cloud   = hourly.Variables(2).ValuesAsNumpy()
    direct  = hourly.Variables(3).ValuesAsNumpy()
    diffuse = hourly.Variables(4).ValuesAsNumpy()

    dates = pd.date_range(
        start=pd.to_datetime(hourly.Time(),    unit="s", utc=True),
        end  =pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq =pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    df_dep = pd.DataFrame({
        "date": dates,
        "temperature_2m": temp,
        "wind_speed_10m": wind,
        "cloud_cover": cloud,
        "direct_radiation": direct,
        "diffuse_radiation": diffuse,
        "dep_code": dep["code"],
        "dep_name": dep["name"],
        "lat": dep["lat"],
        "lon": dep["lon"],
    })

    all_dfs.append(df_dep)

df_idf = pd.concat(all_dfs, ignore_index=True)

output_file = "weather_idf_departements_2024.csv"
df_idf.to_csv(output_file, index=False)

print(f" Données sauvegardées dans '{output_file}'")
print(df_idf.head())


 Récupération des données météo pour Paris (2024)...
 Récupération des données météo pour Hauts-de-Seine (2024)...
 Récupération des données météo pour Seine-Saint-Denis (2024)...
 Récupération des données météo pour Val-de-Marne (2024)...
 Récupération des données météo pour Essonne (2024)...
 Récupération des données météo pour Yvelines (2024)...
 Récupération des données météo pour Val-d'Oise (2024)...
 Récupération des données météo pour Seine-et-Marne (2024)...
 Données sauvegardées dans 'weather_idf_departements_2024.csv'
                       date  temperature_2m  wind_speed_10m  cloud_cover  \
0 2024-01-01 00:00:00+00:00            7.31       26.874910         35.0   
1 2024-01-01 01:00:00+00:00            7.31       25.570139         51.0   
2 2024-01-01 02:00:00+00:00            7.16       25.704100         24.0   
3 2024-01-01 03:00:00+00:00            6.96       25.199997         40.0   
4 2024-01-01 04:00:00+00:00            6.81       24.774696         47.0   

   direct